In [1]:
import random
import numpy as np

import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from fiftyone import ViewField as F

print(fo.__version__)

0.20.1


In [18]:
dsetname = 'qs-eval'
if fo.dataset_exists(dsetname):
    fo.delete_dataset(dsetname)
    
ds = foz.load_zoo_dataset('quickstart',dataset_name=dsetname)
ds.persistent=True

Dataset already downloaded
Loading 'quickstart'
 100% |█████████████████| 200/200 [2.0s elapsed, 0s remaining, 99.4 samples/s]          
Dataset 'qs-eval' created


In [19]:
ds.rename_sample_field('ground_truth','labels')
v = ds.values('predictions')
ds.rename_sample_field('predictions','model_rn50_00')
for s in ds:
    dets0 = s.model_rn50_00
    dets1 = dets0.copy()
    dets2 = dets0.copy()
    s['model_rn75_00'] = dets1
    s['model_rn75_01'] = dets2
    s.save()

In [26]:
fob.compute_similarity(
    ds,
    model="clip-vit-base32-torch",
    brain_key="img_sim",
)

# Index ground truth objects by similarity
fob.compute_similarity(
    ds,
    patches_field="labels",
    model="resnet18-imagenet-torch",
    brain_key="gt_sim",
)

fob.compute_uniqueness(ds)

Computing embeddings...
 100% |█████████████████| 200/200 [9.4s elapsed, 0s remaining, 19.5 samples/s]      
Computing patch embeddings...


/home/allen/venv/chat/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/allen/venv/chat/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 100% |█████████████████| 200/200 [1.4m elapsed, 0s remaining, 2.6 samples/s]       
Computing embeddings...
 100% |█████████████████| 200/200 [1.2s elapsed, 0s remaining, 171.3 samples/s]         
Computing uniqueness...
Uniqueness computation complete


In [27]:
models = ['model_rn50_00','model_rn75_00','model_rn75_01']
for m in models:
    res = ds.evaluate_detections(
        m,
        gt_field="labels",
        eval_key=f"eval_{m}",
    )
    res = ds.evaluate_detections(
        m,
        gt_field="labels",
        eval_key=f"eval_{m}_iou08",
        iou=0.8,
    )
    res = ds.evaluate_detections(
        m,
        gt_field="labels",
        eval_key=f"eval_{m}_oi",
        method='open-images',
    )

Evaluating detections...
 100% |█████████████████| 200/200 [10.5s elapsed, 0s remaining, 13.8 samples/s]      
Evaluating detections...
 100% |█████████████████| 200/200 [12.1s elapsed, 0s remaining, 12.3 samples/s]      
Evaluating detections...
 100% |█████████████████| 200/200 [27.0s elapsed, 0s remaining, 2.5 samples/s]       
Evaluating detections...
 100% |█████████████████| 200/200 [31.6s elapsed, 0s remaining, 5.9 samples/s]       
Evaluating detections...
 100% |█████████████████| 200/200 [25.0s elapsed, 0s remaining, 7.3 samples/s]       
Evaluating detections...
 100% |█████████████████| 200/200 [16.8s elapsed, 0s remaining, 9.1 samples/s]       
Evaluating detections...
 100% |█████████████████| 200/200 [11.5s elapsed, 0s remaining, 12.8 samples/s]      
Evaluating detections...
 100% |█████████████████| 200/200 [16.9s elapsed, 0s remaining, 6.6 samples/s]      
Evaluating detections...
 100% |█████████████████| 200/200 [30.0s elapsed, 0s remaining, 5.4 samples/s]       


In [36]:
import random
import datetime
import uuid

In [46]:
# random metadata
for s in ds:
    s['datetime'] = datetime.datetime.fromtimestamp(random.randint(1,1000))
    s['weather'] = random.choice(('cloudy','sunny','raining','hailing','snowing','clear','windy'))
    s['weather_note'] = random.choice(('cloudy','sunny','raining','hailing','snowing','clear','windy'))
    s['zipcode'] = random.choice(('07920','07921','74054','60540','60563','60562'))
    s['username'] = random.choice(('joe','jane','alex','trish','moe','larry','sheila'))
    s['class'] = random.randint(1,10)
    s['stat_train'] = random.random()*100
    s['uuid'] = str(uuid.uuid4())
    s.save()
ds.save()
print(ds.name)
print(ds.persistent)

qs-eval
True


# ---- random working stuff below ----

In [ ]:
CLSES = [str(x) for x in range(5)]

def jitter(val):
    if random.random() < 0.10:
        return random.choice(CLSES)

    return val

def newval():
    return random.choice(CLSES)

def newval_logits():
    logits = [random.random() for c in CLSES]
    lbl = CLSES[np.argmax(logits)]
    conf = random.random()
    return lbl, conf, logits
    

In [ ]:
#vals_gt = [newval() for s in ds]
#vals_pd = [jitter(x) for x in vals_gt]

vals_gt = [newval_logits() for s in ds]
vals_pd = [newval_logits() for s in ds]
vals_pd2 = [newval_logits() for s in ds]

vals_gt = [fo.Classification(label=l,confidence=c,logits=log) for l,c,log in vals_gt]
vals_pd = [fo.Classification(label=l,confidence=c,logits=log) for l,c,log in vals_pd]
vals_pd2 = [fo.Classification(label=l,confidence=c,logits=log) for l,c,log in vals_pd2]

ds.set_values('cls_gt',vals_gt)
ds.set_values('cls_pd',vals_pd)
ds.set_values('cls_pd2',vals_pd2)

In [ ]:
ds.save()
print(ds)
s=ds.first()

In [ ]:
# clsification eval: 
results = ds.evaluate_classifications(
    "cls_pd",
    gt_field="cls_gt",
    eval_key="eval_cls",
)

results = ds.evaluate_classifications(
    "cls_pd2",
    gt_field="cls_gt",
    eval_key="eval_cls2",
)


In [ ]:
# topk
results = ds.evaluate_classifications(
    "cls_pd",
    gt_field="cls_gt",
    eval_key="eval_cls_topk",
    method="top-k",
    classes=CLSES,
    k=3)

results = ds.evaluate_classifications(
    "cls_pd2",
    gt_field="cls_gt",
    eval_key="eval_cls2_topk",
    method="top-k",
    classes=CLSES,
    k=3)

In [ ]:
fob.compute_hardness(ds,'cls_pd')

In [ ]:
ifos = [ds.get_evaluation_info(x) for x in ds.list_evaluations()]
print(ifos)

In [ ]:
list_evaluations
i=ds.get_evaluation_info()
#h=ds.has_evaluaion
load_evaluation_results
laod_evaluation_view
to_evaluation_pathes

### test 1: multiple pred fields


In [ ]:
ds = foz.load_zoo_dataset('quickstart')

In [ ]:
for s in ds:
    dets = s.predictions.detections.copy()
    s['new_preds'] = fo.Detections(detections=dets)
    s.save()
print(ds)

In [ ]:
res0 = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_preds",
)
res1 = ds.evaluate_detections(
    "new_preds",
    gt_field="ground_truth",
    eval_key="eval_new",
)

In [ ]:
print(ds.list_evaluations())
ds.name='ds_chat_2detpreds'
ds.persistent=True

### test 2: multiple gt fields


In [1]:

ds = foz.load_zoo_dataset('quickstart')
for s in ds:
    dets = s.ground_truth.detections.copy()
    s['gt_new'] = fo.Detections(detections=dets)
    s.save()
print(ds)
res0 = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval",
)
res1 = ds.evaluate_detections(
    "predictions",
    gt_field="gt_new",
    eval_key="eval_newgt",
)
print(ds.list_evaluations())
ds.name='ds_chat_2detGTs'
ds.persistent=True

Dataset already downloaded
Loading 'quickstart'
 100% |█████████████████| 200/200 [2.1s elapsed, 0s remaining, 98.0 samples/s]          
Dataset 'quickstart' created
Name:        quickstart
Media type:  image
Num samples: 200
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    uniqueness:   fiftyone.core.fields.FloatField
    predictions:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    gt_new:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Evaluating detections...
 100% |█████████████████| 200/200 [10.7s elapsed, 0s remaining, 13.6 samples/s]  

### test3: multiple IOU thresholds

In [2]:
ds = foz.load_zoo_dataset('quickstart')
res0 = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_det_normal",
    iou=0.6,
)
res1 = ds.evaluate_detections(
    "predictions",
    gt_field="ground_truth",
    eval_key="eval_det_tight",
    iou=0.9,
)
print(ds.list_evaluations())
ds.name='ds_chat_2detIOUs'
ds.persistent=True

Dataset already downloaded
Loading 'quickstart'
 100% |█████████████████| 200/200 [2.0s elapsed, 0s remaining, 100.4 samples/s]         
Dataset 'quickstart' created
Evaluating detections...
 100% |█████████████████| 200/200 [10.6s elapsed, 0s remaining, 14.0 samples/s]      
Evaluating detections...
 100% |█████████████████| 200/200 [11.7s elapsed, 0s remaining, 12.7 samples/s]      
['eval_det_normal', 'eval_det_tight']


In [4]:
fo.list_datasets()

['ds_al_chat',
 'ds_chat_2detGTs',
 'ds_chat_2detIOUs',
 'ds_chat_2detpreds',
 'quickstart-video']

### video

In [5]:
import random
import numpy as np

In [33]:
DSETNAME = 'qsv-test00'
if fo.dataset_exists(DSETNAME):
    fo.delete_dataset(DSETNAME)
    
dsv = foz.load_zoo_dataset('quickstart-video',dataset_name=DSETNAME)
dsv.compute_metadata()
dsv.ensure_frames()
dsv.persistent=True

dets = dsv.values('frames.detections')
dsv.set_values('frames.dets_mnet',dets)

dsv.delete_frame_field('frames.detections')



Dataset already downloaded
Loading 'quickstart-video'
 100% |███████████████████| 10/10 [4.4s elapsed, 0s remaining, 2.3 samples/s]      
Dataset 'qsv-test00' created
Computing metadata...
 100% |███████████████████| 10/10 [224.2ms elapsed, 0s remaining, 44.6 samples/s] 


In [34]:
# tags
set = np.random.randint(0,2,len(dsv),dtype=np.bool)
dsv[set].tag_samples('Intruder')
set = np.random.randint(0,2,len(dsv),dtype=np.bool)
dsv[set].tag_samples('Safe')

# classes
CLASSES = ['sunny', 'rainy', 'hailing']
num_class = len(CLASSES)
for s in dsv.iter_samples(autosave=True):
    icls = np.random.randint(0,num_class)
    conf = np.random.random()
    c = fo.Classification(label=CLASSES[icls],confidence=conf)
    s['weather'] = c
    
# tempdets
CLASSES = ['vehicle','person','animal']
for s in dsv.iter_samples(autosave=True):
    nf = s.metadata.total_frame_count
    icls = np.random.randint(0,num_class)
    conf = np.random.random()
    f0 = np.random.randint(0,nf)
    f1 = np.random.randint(0,nf)
    if f0 > f1:
        tmp = f0
        f0 = f1
        f1 = tmp
    s['objects'] = fo.TemporalDetection(label=CLASSES[icls],
                                        support=[f0,f1],
                                        confidence=conf)
    
# frame classes
for s in dsv.iter_samples(autosave=True):
    for f,frm in s.frames.items():
        if np.random.random()>0.95:
            icls = np.random.randint(0,num_class)
            conf = np.random.random()
            c = fo.Classification(label=CLASSES[icls],confidence=conf)
            frm['intruder'] = c
            frm.save()
                                    
    
    
    
    

/tmp/ipykernel_13427/1608527792.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  set = np.random.randint(0,2,len(dsv),dtype=np.bool)
/tmp/ipykernel_13427/1608527792.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  set = np.random.randint(0,2,len(dsv),dtype=np.bool)
